This is a mixed document of notes, calculating results and source code, written with jupyter notebook in python 3.7.\
Let's come straight forward to give a grid construct of lattices each marks a spin

In [ ]:
import numpy as np
from numpy import random
import matplotlib as plt
import matplotlib.pyplot as pltt
import pandas as pd

Here are some parameters

In [ ]:
J=1
Tmin=0.1
Tmax=5
interations=13000
mc=700
scale=20
#initializing a grid with spins all up/randomized
#grid=np.ones([scale,scale], int)
grid = np.random.randint(0, 2, [scale, scale]) * 2 - 1

Now define some useful functions using periodic boundary conditions

In [ ]:
#initializing a grid with spins all up/randomized
#grid=np.ones([scale,scale], int)
def set():
    global grid
    grid = np.random.randint(0, 2, [scale, scale]) * 2 - 1
    return 1

#finding side spins according to periodic boudary conditions
def left(x, y):
    global grid
    spin=grid   
    if x < 0.6:
        return spin[scale-1,y]
    else:
        return spin[x-1, y]

def right(x,y):
    global grid
    spin=grid
    if x > scale-1.4 :
        return spin[0,y]
    else:
        return spin[x+1,y]
    
def up(x,y):
    global grid
    spin=grid
    if y > scale-1.4:
        return spin[x,0]
    else:
        return spin[x,y+1]
    
def down(x,y):
    spin=grid
    if y < 0.4:
        return spin[x,scale-1]
    else:
        return spin[x,y-1]    

In [ ]:
#functions calculating physical properties
#iteraction energy of a given spin
def E_unit(x,y):
    global grid
    spin=grid
    E_unit= -J * spin[x,y] * (left(x,y) + right(x,y) + up(x,y) + down(x,y))
    return E_unit

#energy change by flipping it
def deltaE(x,y):
    return -2 * E_unit(x,y)

#total energy of the system
def E_tot():
    E = 0
    for i in range(0,scale):
        for j in range(0,scale):
            E = E + E_unit(i,j)
    E = 0.5 * E
    return E

#total magnetization of the system
def M_tot():
    M=0
    for i in range(0,scale):
        for j in range(0,scale):
            M = M + grid[i,j]
    return M    

Introducing the Metropolis algothrim in the follow function

In [ ]:
#a complete Metrpolis loop
def metroloop():
    for i in range(0,interations):
        flip()
    return 1

#define a random flip
def flip():
    global E
    global M
    x = np.random.randint(0, scale)
    y = np.random.randint(0, scale)
    if deltaE(x,y) < 0 :
        grid[x,y] = -grid[x,y]
        E = E - deltaE(x,y)
        M = M + 2 * grid[x,y]
    else:
        if np.random.rand() < np.exp(-deltaE(x,y) / T):
            grid[x,y] = -grid[x,y]
            E = E - deltaE(x,y)    
            M = M + 2 * grid[x,y]
    return 1

Now let's try to run a loop to verify our code under low temperature

In [ ]:
set()
E=E_tot()
M=M_tot()
T=0.2
metroloop()
pltt.imshow(grid)
pltt.show()

The spins are quiet orderly alligned, which matches pretty well with the low temperature limit.\
With almost everything under preparation, we can now start to write the main program.

In [ ]:
#get enough samples to calculate average values of E&M
def sampling():
    for i in range(0,mc):
        global Esum
        global Msum
        global Esqr
        E = E_tot()
        M = M_tot()
        metroloop()
        Esum = Esum + E
        Esqr = Esqr + E ** 2
        Msum = Msum + abs( M )
    return 1

In [ ]:
#main programe
import csv
for T in np.arange(Tmin,Tmax,0.1):
    Esum = 0
    Esqr = 0
    Msum = 0
    set()
    sampling() #do Monte Carlo simulation
    E_avr = Esum / (400 * mc)
    Esqr_avr = Esqr / (400 * 400 * mc)
    M_avr = Msum / (400 * mc)
    #putting data into a csv
    out = open('D:\\Eavr.csv','a',newline='')
    csv_write = csv.writer(out)
    csv_write.writerow(str(E_avr))
    out = open('D:\\Esqr.csv','a',newline='')
    csv_write = csv.writer(out)
    csv_write.writerow(str(Esqr_avr))
    out = open('D:\\Mavr.csv','a',newline='')
    csv_write = csv.writer(out)
    csv_write.writerow(str(M_avr))

Start plotting the figures

In [ ]:
M_ = pd.read_csv('D:\\Mavr.csv')
Mvalue = M_.values
E_ = pd.read_csv('D:\\Eavr.csv')
Evalue = E_.values
Esqr_ = pd.read_csv('D:\\Esqr.csv')
Esqrvalue = Esqr_.values
C = np.random.random(size=(46,1))
for i in range(0,46):
    C[i,0] = (Evalue[i+1,0] - Evalue[i,0]) / 0.1
t1=np.arange(0.1,4.71,0.1)
pltt.plot(t1,Evalue,'yD-',lw=2,alpha=0.8)
pltt.title('Energy per lattice')
pltt.xlabel('Temperature')
pltt.ylabel('Energy')
pltt.show()

In [ ]:
t2=np.arange(0.1,4.81,0.1)
pltt.plot(t2,Mvalue,'gD-',lw=2,alpha=0.8)
pltt.title('Magnetisation per lattice')
pltt.xlabel('Temperature')
pltt.ylabel('Magnetisation')
pltt.show()

In [ ]:
t3=np.arange(0.15,4.66,0.1)
pltt.plot(t3,C,'mD-',lw=2,alpha=0.8)
pltt.title('Specific Heat Capacity per lattice')
pltt.xlabel('Temperature')
pltt.ylabel('Heat Capacity')
pltt.show()

All of the figures above shows an ABRUPT change of physical properties near $T=2.2\epsilon / k_B$, which marks the PHASE TRANSITION.